In [1]:
import pip

package_names = ['lightgbm', 'awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)


from setup import *

NumExpr defaulting to 4 threads.
////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
target :::::::::  target_desembolso_f2m_mayor_30_menor_180


In [3]:
# imports
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri
import awswrangler as wr
import pandas as pd
import sagemaker
import boto3


print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)
print("sufijo: ", sufijo)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-27 15:49:35.272460
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


In [4]:
print("uri_split: ", uri_split)
print("uri_train_xgb: ", uri_train_xgb)
print("uri_valid_xgb: ", uri_valid_xgb)
print("uri_models: ", uri_models)

uri_split:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split
uri_train_xgb:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/train.csv
uri_valid_xgb:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/validation.csv
uri_models:  s3://sagemaker-us-east-1-058528764918/vpc/propension/models


In [5]:
valid = pd.read_csv(uri_valid_xgb)
print(valid.shape)

del valid
gc.collect()

(2153910, 20)


700

### MODELADO

In [6]:
container = get_image_uri(region, 'xgboost', '0.90-1')
role = sagemaker.get_execution_role()  

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m5.4xlarge',
                                    output_path=uri_models,
                                    sagemaker_session=sess)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3
Defaulting to only supported image scope: cpu.


In [7]:
xgb.set_hyperparameters(eval_metric = 'auc',   #'rmse',
                        objective = 'binary:logistic', #'reg:squarederror',          ---ojo se cambian ya que ahora el objetivo es clasiificacion
                        seed=SEED,
                        early_stopping_rounds = 25,
                        num_round = 1000,
                        #booster='dart'
                       )

hyperparameter_ranges = {
    #'scale_pos_weight': ContinuousParameter(0.015, 0.035),
    'colsample_bytree': ContinuousParameter(0.1, 0.9),
    'colsample_bynode': ContinuousParameter(0.1, 0.9),
    'colsample_bylevel': ContinuousParameter(0.1, 0.9),
    'max_depth': IntegerParameter(2, 7),                                       #### profundidad de los arboles
    'eta': ContinuousParameter(0.01, 0.5),                                     #### tasa de aprendizaje
    #'rate_drop': ContinuousParameter(0.1, 0.5),
    #'skip_drop': ContinuousParameter(0.1, 0.5),
    #'one_drop': IntegerParameter(0, 1),
    #'sample_type': CategoricalParameter(['uniform', 'weighted']),
    #'normalize_type': CategoricalParameter(['tree', 'forest']),
    #'tree_method': CategoricalParameter(['approx', 'hist']),
    'subsample': ContinuousParameter(0.1, 0.9),
    'gamma': ContinuousParameter(0, 10),
    'max_delta_step': IntegerParameter(0, 10),
    'min_child_weight': IntegerParameter(0, 120)
}

In [8]:
objective_metric_name = 'validation:auc'

In [9]:
tuner = HyperparameterTuner(
    xgb,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=45,
    max_parallel_jobs=9,
    objective_type='Maximize',   #Minimize OJO!!! era mínimo ya que mientras menor RMSE mejor modelo de regresion, acá cambia /// mayor AUc mejor modelo 
    base_tuning_job_name='xgboost-vpc-7',
    early_stopping_type='Auto'
)

In [10]:
#help(tuner)

In [11]:
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data=uri_train_xgb, 
    content_type='csv'
)   # OJO!! revisar cambio por la version

s3_input_validation = sagemaker.inputs.TrainingInput(
    s3_data=uri_valid_xgb, 
    content_type='csv'
) # OJO!! revisar cambio por la version

In [ ]:
tuner.fit(
    {
        'train': s3_input_train, 
        'validation': s3_input_validation
    }, 
    include_cls_metadata=False
)

Defaulting to the only supported framework/algorithm version: latest.
Ignoring unnecessary instance type: None.
Creating hyperparameter tuning job with name: xgboost-vpc-7-210927-1550
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [ ]:
print(tuner.latest_tuning_job.name)

xgboost-vpc-7-210927-1550


In [ ]:
tuner.best_training_job()

'xgboost-vpc-7-210927-1550-041-301a74a7'

In [ ]:
tuner.describe()

{'HyperParameterTuningJobName': 'xgboost-vpc-7-210927-1550',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:058528764918:hyper-parameter-tuning-job/xgboost-vpc-7-210927-1550',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Maximize',
   'MetricName': 'validation:auc'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 45,
   'MaxParallelTrainingJobs': 9},
  'ParameterRanges': {'IntegerParameterRanges': [{'Name': 'max_depth',
     'MinValue': '2',
     'MaxValue': '7',
     'ScalingType': 'Auto'},
    {'Name': 'max_delta_step',
     'MinValue': '0',
     'MaxValue': '10',
     'ScalingType': 'Auto'},
    {'Name': 'min_child_weight',
     'MinValue': '0',
     'MaxValue': '120',
     'ScalingType': 'Auto'}],
   'ContinuousParameterRanges': [{'Name': 'colsample_bytree',
     'MinValue': '0.1',
     'MaxValue': '0.9',
     'ScalingType': 'Auto'},
    {'Name': 'colsample_bynode',
     'MinValue': '0.1',
     'MaxValue': 

In [ ]:
nombre_exportado = HyperparameterTuner.attach(tuner.latest_tuning_job.name).best_training_job()
nombre_exportado

'xgboost-vpc-7-210927-1550-041-301a74a7'

In [ ]:
from sagemaker.estimator import Estimator, Transformer
model = Estimator.attach(nombre_exportado)


2021-09-27 16:28:31 Starting - Preparing the instances for training
2021-09-27 16:28:31 Downloading - Downloading input data
2021-09-27 16:28:31 Training - Training image download completed. Training in progress.
2021-09-27 16:28:31 Uploading - Uploading generated training model
2021-09-27 16:28:31 Completed - Training job completed


In [ ]:
tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name)
tuner_metrics

<sagemaker.HyperparameterTuningJobAnalytics for xgboost-vpc-7-210927-1550>

In [ ]:
results = tuner_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)
results.head(50)

,colsample_bylevel,colsample_bynode,colsample_bytree,eta,gamma,max_delta_step,max_depth,min_child_weight,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
5,0.454127,0.517129,0.825732,0.428295,0.947134,6.0,7.0,60.0,0.716676,xgboost-vpc-7-210927-1550-041-301a74a7,Completed,0.760952,2021-09-27 16:24:03+00:00,2021-09-27 16:28:31+00:00,268.0
7,0.858961,0.752139,0.683609,0.487302,1.022207,9.0,7.0,20.0,0.790455,xgboost-vpc-7-210927-1550-039-2bb4a862,Completed,0.760345,2021-09-27 16:20:02+00:00,2021-09-27 16:25:56+00:00,354.0
8,0.858961,0.752139,0.699609,0.487302,1.022207,9.0,7.0,22.0,0.790455,xgboost-vpc-7-210927-1550-038-2deb164e,Completed,0.760164,2021-09-27 16:19:23+00:00,2021-09-27 16:22:58+00:00,215.0
20,0.603639,0.283067,0.815103,0.258850,3.294546,6.0,7.0,94.0,0.561485,xgboost-vpc-7-210927-1550-026-2085927d,Completed,0.759848,2021-09-27 16:10:51+00:00,2021-09-27 16:16:55+00:00,364.0
21,0.656910,0.578408,0.656363,0.477313,7.349111,5.0,7.0,59.0,0.700904,xgboost-vpc-7-210927-1550-025-359d7875,Completed,0.759697,2021-09-27 16:09:09+00:00,2021-09-27 16:13:27+00:00,258.0
41,0.792898,0.699827,0.626100,0.471547,6.117643,5.0,7.0,118.0,0.655603,xgboost-vpc-7-210927-1550-005-2e6cc7fc,Completed,0.759653,2021-09-27 15:55:55+00:00,2021-09-27 16:02:33+00:00,398.0
11,0.656910,0.578408,0.672363,0.467513,7.349111,5.0,7.0,59.0,0.716904,xgboost-vpc-7-210927-1550-035-051f5082,Completed,0.759650,2021-09-27 16:15:58+00:00,2021-09-27 16:22:00+00:00,362.0
12,0.694215,0.490322,0.333902,0.291529,0.000773,4.0,7.0,4.0,0.854225,xgboost-vpc-7-210927-1550-034-843e4d23,Completed,0.759281,2021-09-27 16:16:12+00:00,2021-09-27 16:23:29+00:00,437.0
1,0.157317,0.647940,0.787206,0.231834,3.159658,7.0,7.0,14.0,0.869248,xgboost-vpc-7-210927-1550-045-962749a5,Completed,0.758379,2021-09-27 16:26:47+00:00,2021-09-27 16:30:59+00:00,252.0
37,0.811480,0.292135,0.359663,0.228813,2.019983,4.0,7.0,71.0,0.588674,xgboost-vpc-7-210927-1550-009-8ca63e4e,Completed,0.758224,2021-09-27 15:53:42+00:00,2021-09-27 15:59:34+00:00,352.0


## GUARDADO DE RESULTADOS

In [ ]:
print(save_results)

resuts_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv


In [ ]:
results.to_csv(save_results, index=False)